# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [58]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv(r'C:\Users\JEFFEY MARKUS\production\05_src\data\adult\adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [59]:
X = adult_dt.drop(columns='income')
Y = adult_dt['income']

from sklearn.model_selection import train_test_split

# Splitting the data into test and train sets.
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

In [60]:
print("\033[92mRandom state is used to control the randomness of the data splitting process.\033[0m")
print("\033[92mBy specifying a random state, we are setting a seed for the random number generator, ensuring that the train-test split is reproducible every time we run our code.\n\033[0m")

print("\033[1m\033[92mWhy is it Useful?\033[0m\n")
print("\033[92m1. Reproducibility: Setting a random state makes sure that if we run the same code again, we get the same split of the data into training and testing sets.\033[0m")
print("\033[92mThis is crucial for debugging, sharing results, and consistent model performance during experiments.\n\033[0m")

print("\033[92m2. Consistency in Model Comparison: When we compare different models or different configurations of the same model, it is beneficial to have a consistent train-test split.\033[0m")
print("\033[92mThis consistency ensures that any differences in model performance are due to the changes in the model and not variations in how the data is split.\033[0m")


Random state is used to control the randomness of the data splitting process.
By specifying a random state, we are setting a seed for the random number generator, ensuring that the train-test split is reproducible every time we run our code.

Why is it Useful?

1. Reproducibility: Setting a random state makes sure that if we run the same code again, we get the same split of the data into training and testing sets.
This is crucial for debugging, sharing results, and consistent model performance during experiments.

2. Consistency in Model Comparison: When we compare different models or different configurations of the same model, it is beneficial to have a consistent train-test split.
This consistency ensures that any differences in model performance are due to the changes in the model and not variations in how the data is split.


# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [61]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
data = pd.DataFrame(np.random.randn(100, len(columns)), columns=columns)  # Dummy data; replace with your data

# Separating numerical and categorical columns
numerical_cols = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
categorical_cols = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

# Define transformers for numerical and categorical features
numerical_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('scaler', RobustScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))
])

# Creating the ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# Applying the preprocessing pipeline 
X_transformed = preprocessor.fit_transform(data)


## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [62]:
from sklearn.ensemble import RandomForestClassifier

# Creating the model pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),  # The ColumnTransformer from the previous step
    ('classifier', RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42))  # The classifier
])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [63]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score
import numpy as np

# Defining the performance metrics
scoring = {
    'neg_log_loss': 'neg_log_loss',
    'roc_auc': 'roc_auc',
    'accuracy': 'accuracy',
    'balanced_accuracy': 'balanced_accuracy'
}

# Evaluating the model pipeline using 5-fold cross-validation
cv_results = cross_validate(
    model_pipeline, 
    X, 
    Y, 
    cv=5, 
    scoring=scoring, 
    return_train_score=True
)

# Reporting the training and validation results
print("\nCross-Validation Results:")
for metric in scoring.keys():
    train_mean = np.mean(cv_results[f'train_{metric}'])
    train_std = np.std(cv_results[f'train_{metric}'])
    test_mean = np.mean(cv_results[f'test_{metric}'])
    test_std = np.std(cv_results[f'test_{metric}'])
    print(f"{metric} - Training: {train_mean:.4f} (+/- {train_std:.4f}), Validation: {test_mean:.4f} (+/- {test_std:.4f})")


c:\Users\JEFFEY MARKUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\JEFFEY MARKUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(



Cross-Validation Results:
neg_log_loss - Training: -0.3118 (+/- 0.0017), Validation: -0.3281 (+/- 0.0030)
roc_auc - Training: 0.9254 (+/- 0.0008), Validation: 0.9106 (+/- 0.0034)
accuracy - Training: 0.8653 (+/- 0.0012), Validation: 0.8568 (+/- 0.0043)
balanced_accuracy - Training: 0.7630 (+/- 0.0023), Validation: 0.7507 (+/- 0.0076)


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [64]:
cv_results_df = pd.DataFrame(cv_results)

# Sorting the DataFrame by 'test_neg_log_loss'
sorted_cv_results = cv_results_df.sort_values(by='test_neg_log_loss')

# Displaying the sorted DataFrame
print("Sorted Cross-Validation Results by Test Negative Log Loss:\n")
print(sorted_cv_results)


Sorted Cross-Validation Results by Test Negative Log Loss:

   fit_time  score_time  test_neg_log_loss  train_neg_log_loss  test_roc_auc  \
1  3.632418    0.175431          -0.331249           -0.309713      0.906679   
0  3.129728    0.156102          -0.330795           -0.311253      0.907006   
2  2.931507    0.144618          -0.329205           -0.313765      0.910617   
4  3.292911    0.146331          -0.325861           -0.313799      0.914390   
3  3.131725    0.149254          -0.323481           -0.310539      0.914494   

   train_roc_auc  test_accuracy  train_accuracy  test_balanced_accuracy  \
1       0.926736       0.855651        0.865331                0.747288   
0       0.925341       0.848918        0.865095                0.737764   
2       0.924791       0.858722        0.866790                0.755843   
4       0.924358       0.859951        0.866060                0.759700   
3       0.925640       0.860872        0.863334                0.752904   

   train

Calculate the mean of each metric. 

In [65]:
mean_metrics = cv_results_df.mean()

# Displaying the mean values for each metric
print("Mean values for each metric across the folds:")
print(mean_metrics)


Mean values for each metric across the folds:
fit_time                   3.223658
score_time                 0.154347
test_neg_log_loss         -0.328118
train_neg_log_loss        -0.311814
test_roc_auc               0.910637
train_roc_auc              0.925373
test_accuracy              0.856823
train_accuracy             0.865322
test_balanced_accuracy     0.750700
train_balanced_accuracy    0.763034
dtype: float64


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [66]:
model_pipeline.fit(X_train, Y_train)
Y_pred_proba = model_pipeline.predict_proba(X_test)

# Calculating the required metrics
test_metrics = {
    'neg_log_loss': log_loss(Y_test, Y_pred_proba),
    'roc_auc': roc_auc_score(Y_test, Y_pred_proba[:, 1]),
    'accuracy': accuracy_score(Y_test, model_pipeline.predict(X_test)),
    'balanced_accuracy': balanced_accuracy_score(Y_test, model_pipeline.predict(X_test))
}

# Displaying the test metrics as a dictionary
print("Test Performance Metrics:")
print(test_metrics)

Test Performance Metrics:
{'neg_log_loss': 0.3248655516773495, 'roc_auc': np.float64(0.9101514010597728), 'accuracy': 0.8592486436687481, 'balanced_accuracy': np.float64(0.7513226869137615)}


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

In [69]:
print("\033[92mThe target variable 'income' was recoded to:")
print("  -  1 for >50K ")
print("  -  0 for <=50K ")
print("1.This transformation ensures compatibility with machine learning algorithms that expect numeric targets.")
print("2.Simplifies data processing and avoids the need for later conversions.")
print("3.Removes extra whitespace using 'str.strip()' for consistent data.")
print("4.Makes subsequent modeling steps more efficient by treating 'income' as a clean, binary numeric variable.\033[0m")


The target variable 'income' was recoded to:
  -  1 for >50K 
  -  0 for <=50K 
1.This transformation ensures compatibility with machine learning algorithms that expect numeric targets.
2.Simplifies data processing and avoids the need for later conversions.
3.Removes extra whitespace using 'str.strip()' for consistent data.
4.Makes subsequent modeling steps more efficient by treating 'income' as a clean, binary numeric variable.


## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.